In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pydot
import graphviz
from clr_callback import CyclicLR

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.client import device_lib 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import save_model, load_model

tf.random.set_seed(42)

train_directory = "./data/organized/train/"
val_directory = "./data/organized/val/"
test_directory = "./data/organized/test/"

In [2]:
# Allow Tensorflow to allocate GPU memory as needed, rather than pre-allocating the entire GPU memory at the start of program execution.
# This option allows for better monitoring of system resource utilization.
physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
# ===============================================================================================================
# This function creates training, validation and test datasets using the file structure created in the
# 01_create_train_val_test_directories notebook. 
# ===============================================================================================================
def create_tensorflow_datasets(image_size, train_directory, val_directory, test_directory, batch_size=32):
    
    train_dataset = image_dataset_from_directory(directory = train_directory,
                                                 labels='inferred',
                                                 label_mode = 'int',
                                                 image_size=image_size,
                                                 batch_size=batch_size,
                                                 smart_resize=True)

    val_dataset = image_dataset_from_directory(directory = val_directory,
                                               labels='inferred',
                                               label_mode = 'int',
                                               image_size=image_size,
                                               batch_size=batch_size,
                                               smart_resize=True)

    test_dataset = image_dataset_from_directory(directory = test_directory,
                                                labels = "inferred",
                                                label_mode = "int",
                                                image_size=image_size,
                                                batch_size=batch_size,
                                                smart_resize=True)
    
    return train_dataset, val_dataset, test_dataset

In [4]:
train_dataset, val_dataset, test_dataset = create_tensorflow_datasets(image_size=(520, 520),
                                                                      train_directory=train_directory,
                                                                      val_directory=val_directory,
                                                                      test_directory=test_directory,
                                                                      batch_size=32)

Found 10520 files belonging to 196 classes.
Found 3234 files belonging to 196 classes.
Found 2431 files belonging to 196 classes.


In [5]:
def unfreeze_resnet_conv_layers(model_filepath, resnet_layer = 4, num_layers_to_finetune = 5):
    
    # Load the resnet with already trained classifier attached.
    model = load_model(model_filepath)
    
    # Unfreeze all resnet layers
    model.layers[resnet_layer].trainable = True
    
    # List layer numbers for all convolutional layers in the last block in the resnet 101 model.
    block_5_layers = [343, 347, 350, 351, 355, 359, 362, 366, 370, 373]
    
    # List of layers to tune
    layers_to_tune = block_5_layers[-num_layers_to_finetune:]
    
    # Freeze all layers not in the list above
    layers_to_freeze = [index for index, val in enumerate(model.layers[resnet_layer].layers)]
    layers_to_freeze = [layer for layer in layers_to_freeze if layer not in layers_to_tune]
    
    # Freeze all layers we are not tuning by setting the layers trainable attribute to False.
    for layer_num in layers_to_freeze:
        model.layers[resnet_layer].layers[layer_num].trainable = False
        
    for index, layer_num in enumerate(block_5_layers):
        print("================================")
        print(f"Layer Number: {index}")
        print(f"Layer: {layer_num}")
        print(f"Resnet Layer Num: {model.layers[resnet_layer].layers[layer_num]}")
        print(f"Num Trainable Weights: {len(model.layers[resnet_layer].layers[layer_num].trainable_weights)}")
        print("================================\n")
    
    print(model.summary())
    
    return model

In [6]:
# Initial model with no fine tuning yet
model_filepath = "./trained_models/convnet/first_resnet_lr_decay_ealy_stop/2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_21_TO_40.keras"

model = unfreeze_resnet_conv_layers(model_filepath, resnet_layer = 4, num_layers_to_finetune = 5)

Layer Number: 0
Layer: 343
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9C2610>
Num Trainable Weights: 0

Layer Number: 1
Layer: 347
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9C2D00>
Num Trainable Weights: 0

Layer Number: 2
Layer: 350
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9C9F10>
Num Trainable Weights: 0

Layer Number: 3
Layer: 351
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9CD550>
Num Trainable Weights: 0

Layer Number: 4
Layer: 355
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9CDC10>
Num Trainable Weights: 0

Layer Number: 5
Layer: 359
Resnet Layer Num: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002208E9D2EB0>
Num Trainable Weights: 1

Layer Number: 6
Layer: 362
Resnet Layer Num: <tensorflow.python.keras.layers

In [7]:
model_save_path = "./trained_models/convnet/first_resnet_lr_decay_ealy_stop/2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_41_TO_80.keras"

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                 save_best_only=True,
                                                 monitor="val_loss",
                                                 verbose=1),
             keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.5,
                                               patience=2,
                                               min_lr=5e-7,
                                               verbose=1)]

model.compile(loss = SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=2e-6),
              metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 520, 520, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 520, 520, 3)       0         
_________________________________________________________________
resnet101v2 (Functional)     (None, 17, 17, 2048)      42626560  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0     

In [8]:
history = model.fit(train_dataset,
                    epochs=40,
                    validation_data=val_dataset,
                    callbacks=callbacks)

Epoch 1/40
329/329 [==============================] - 240s 709ms/step - loss: 1.5206 - accuracy: 0.5464 - val_loss: 1.0394 - val_accuracy: 0.6939

Epoch 00001: val_loss improved from inf to 1.03936, saving model to ./trained_models/convnet/first_resnet_lr_decay_ealy_stop\2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_41_TO_80.keras


C:\Users\Braden\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/40
329/329 [==============================] - 229s 694ms/step - loss: 1.4827 - accuracy: 0.5531 - val_loss: 1.0372 - val_accuracy: 0.6991

Epoch 00002: val_loss improved from 1.03936 to 1.03719, saving model to ./trained_models/convnet/first_resnet_lr_decay_ealy_stop\2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_41_TO_80.keras
Epoch 3/40
329/329 [==============================] - 228s 694ms/step - loss: 1.4746 - accuracy: 0.5571 - val_loss: 1.0295 - val_accuracy: 0.7007

Epoch 00003: val_loss improved from 1.03719 to 1.02949, saving model to ./trained_models/convnet/first_resnet_lr_decay_ealy_stop\2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_41_TO_80.keras
Epoch 4/40
329/329 [==============================] - 229s 695ms/step - loss: 1.4619 - accuracy: 0.5645 - val_loss: 1.0257 - val_accuracy: 0.7032

Epoch 00004: val_loss improved from 1.02949 to 1.02568, saving model to ./trained_models/convnet/first_resnet_lr_decay_ealy_stop\2021_07_20-13_14_36_resnet101_arch

In [9]:
df = pd.DataFrame(history.history)

df.to_csv("./model_histories/2021_07_20-13_14_36_resnet101_arch1_lrdecay_FINE_TUNE_41_TO_80_HISTORY.csv", index = False)

In [10]:
df.tail()

,loss,accuracy,val_loss,val_accuracy,lr
35,1.400571,0.573479,0.988342,0.712740,5.000000e-07
36,1.365050,0.581654,0.986367,0.713358,5.000000e-07
37,1.379661,0.577947,0.986872,0.713358,5.000000e-07
38,1.371359,0.580894,0.985397,0.714286,5.000000e-07
39,1.377250,0.582414,0.988127,0.713358,5.000000e-07
